In [0]:
from pyspark.sql.functions import count, countDistinct, col, avg, max, datediff, current_date, first

In [0]:
photos_df = spark.table("nasa_rover_gold.photos")
mission_manifest_df = spark.table("nasa_rover_gold.mission_manifest")
cameras_df = spark.table("nasa_rover_gold.camera")

### Photo Summary Table

In [0]:
photos_summary_df = photos_df.groupBy("sol").agg(
    count("photo_id").alias("photo_count"),
    countDistinct("camera_id").alias("camera_count"),
    first("rover_id").alias("rover_id")
).orderBy("sol")

photos_summary_df.display()

### Camera Usage Table

In [0]:
camera_usage_df = photos_df.groupBy("camera_id").agg(
    count("photo_id").alias("photo_count")
).join(
    cameras_df,
    photos_df["camera_id"] == cameras_df["camera_id"], 
    how="left"
).select(
    photos_df["camera_id"],
    col("camera_name").alias("camera_name"),
    col("rover_id"),
    col("photo_count")
).orderBy(col("photo_count").desc())

camera_usage_df.display()

Databricks visualization. Run in Databricks to view.

### Rover Activity Table

In [0]:
rover_activity_df = photos_df.groupBy("rover_id").agg(
    count("photo_id").alias("total_photos"),
    avg("sol").alias("average_sol_photos"),
    max("sol").alias("max_sol")
).join(mission_manifest_df, on="rover_id", how="left").select(
    col("rover_id"),
    col("rover_name"),
    col("total_photos"),
    col("average_sol_photos"),
    col("max_sol")
).orderBy(col("total_photos").desc())

rover_activity_df.display()

Databricks visualization. Run in Databricks to view.

### Camera Efficiency Metrics

In [0]:
camera_efficiency_df = photos_df.groupBy("camera_id", "rover_id").agg(
    count("photo_id").alias("photo_count"),
    (count("photo_id") / countDistinct("sol")).alias("efficiency_score")
).join(cameras_df, photos_df["camera_id"] == cameras_df["camera_id"], how="left") \
.select(
    photos_df["camera_id"],
    cameras_df["camera_name"].alias("camera_name"),
    photos_df["rover_id"],
    "photo_count",
    "efficiency_score"
).orderBy(col("efficiency_score").desc())

camera_efficiency_df.display()


Databricks visualization. Run in Databricks to view.

### Rover Deployment Timeline

In [0]:
# Calculate rover deployment timeline
rover_deployment_df = mission_manifest_df.select(
    "rover_id",
    "rover_name",
    "rover_landing_date",
    "rover_launch_date",
    "rover_status",
    datediff(current_date(), col("rover_landing_date")).alias("mission_duration")
).orderBy("rover_landing_date")

rover_deployment_df.display()

Databricks visualization. Run in Databricks to view.

### Photo Distribution by Camera and Rover



In [0]:
photo_distribution_df = photos_df.groupBy("rover_id", "camera_id").agg(
    count("photo_id").alias("photo_count")
).join(
    cameras_df, 
    photos_df["camera_id"] == cameras_df["camera_id"], 
    how="left"
).select(
    photos_df["rover_id"],
    photos_df["camera_id"],
    cameras_df["camera_name"],
    "photo_count"
).orderBy(col("photo_count").desc())

photo_distribution_df.display()

In [0]:
#Write the specific tables to the delta lake.
photos_summary_df.write.format("delta").mode("overwrite").saveAsTable("nasa_rover_gold.photos_summary")
rover_deployment_df.write.format("delta").mode("overwrite").saveAsTable("nasa_rover_gold.rover_deployment")
rover_activity_df.write.format("delta").mode("overwrite").saveAsTable("nasa_rover_gold.rover_activity")
photo_distribution_df.write.format("delta").mode("overwrite").saveAsTable("nasa_rover_gold.photo_distribution")
camera_efficiency_df.write.format("delta").mode("overwrite").saveAsTable("nasa_rover_gold.camera_efficiency")
camera_usage_df.write.format("delta").mode("overwrite").saveAsTable("nasa_rover_gold.camera_usage")